<a href="https://colab.research.google.com/github/secutron/TesTime/blob/main/FixMatch_ignite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install requirements:

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  # VERSION = "nightly"
  VERSION = "20200607"
  # VERSION = "2020060f"

  !pip install gsutil

  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd


     |████████████████████████████████| 2.9MB 7.0MB/s 
     |████████████████████████████████| 143kB 30.8MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 1.4MB 37.0MB/s 
     |████████████████████████████████| 112kB 38.9MB/s 
     |████████████████████████████████| 3.2MB 38.8MB/s 
  Created wheel for gsutil: filename=gsutil-4.63-cp37-none-any.whl size=3693670 sha256=6d423f2a4d87a2f098ecde36ab84205828e6617347f53cc62f3cce4b78cc70b0
  Stored in directory: /root/.cache/pip/wheels/0d/a1/dc/7da3f9e06320dbfe7989f9d036bd396eff3dbf25eb67cf0e30
  Created wheel for gcs-oauth2-boto-plugin: filename=gcs_oauth2_boto_plugin-2.7-cp37-none-any.whl size=23218 sha256=69b0f2ff216038a112c477e2cd2282e1a6d1e157e127602cc3d1613404c02f11
  Stored in directory: /root/.cache/pip/wheels/2e/6b/7c/bd86832ceb17e0ae3d362c44f461832452eeaacddfcf9128ee
  Created w

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 133kB 14.2MB/s 
     |████████████████████████████████| 163kB 31.3MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=d8a842e554dac792d97e78e91e4851f2f20ab748d46bfa1154b11b67c4c58726
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=043981020fb032b0d58a04c53534c05a922d894391bb9b848b40bc92421df456
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
Uninstalling Pillow-7.1.2:
  Successfully uninstalled Pillow-7.1.2
     |████████████████████████████████| 634kB 5.4MB/s 
  Created wheel for pillow-s

### Download dataset:

In [2]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("./cifar10", train=True, download=True);

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


### Get the code:

In [3]:
!git clone https://github.com/secutron/FixMatch-pytorch/
#!git clone https://github.com/vfdev-5/FixMatch-pytorch/


Cloning into 'FixMatch-pytorch'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 248 (delta 139), reused 164 (delta 76), pack-reused 0
Receiving objects: 100% (248/248), 78.18 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (139/139), done.


#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [4]:
# !wandb login <token>

In [4]:
%cd FixMatch-pytorch/

/content/FixMatch-pytorch


In [5]:
import torch

import ignite.distributed as idist
from ignite.engine import Events
from ignite.utils import manual_seed, setup_logger

import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig

import utils
import trainers
from ctaugment import get_default_cta, OPS, interleave, deinterleave


sorted_op_names = sorted(list(OPS.keys()))

In [7]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize(config_path="config"):
    cfg=compose(config_name="fixmatch.yaml")
    print(OmegaConf.to_yaml(cfg))

dataflow:
  name: cifar10
  data_path: /tmp/cifar10
  batch_size: 64
  num_workers: 12
solver:
  num_epochs: 1024
  epoch_length: 128
  checkpoint_every: 500
  validate_every: 1
  resume_from: null
  optimizer:
    cls: torch.optim.SGD
    params:
      lr: 0.03
      momentum: 0.9
      weight_decay: 0.0001
      nesterov: false
  supervised_criterion:
    cls: torch.nn.CrossEntropyLoss
  lr_scheduler:
    cls: torch.optim.lr_scheduler.CosineAnnealingLR
    params:
      eta_min: 0.0
      T_max: null
  unsupervised_criterion:
    cls: torch.nn.CrossEntropyLoss
    params:
      reduction: none
ssl:
  num_train_samples_per_class: 25
  confidence_threshold: 0.95
  lambda_u: 1.0
  mu_ratio: 7
  cta_update_every: 1
name: fixmatch
seed: 543
debug: false
model: resnet18
num_classes: 10
ema_decay: 0.999
distributed:
  backend: null
  nproc_per_node: null
online_exp_tracking:
  wandb: false



/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'ssl/cta_pseudo': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'solver/default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'dataflow/cifar10': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""


In [8]:
cfg.distributed.backend = "xla-tpu"
cfg.distributed.nproc_per_node = 8
cfg.online_exp_tracking.wandb = "true"
cfg.solver.num_epochs = 50
cfg.ssl.confidence_threshold = 0.7
cfg.ema_decay = 0.9
cfg.ssl.cta_update_every = 15
cfg.solver.optimizer.params.lr = 0.1
#cfg.solver.optimizer.param_groups.lr = 0.1


print(OmegaConf.to_yaml(cfg))

dataflow:
  name: cifar10
  data_path: /tmp/cifar10
  batch_size: 64
  num_workers: 12
solver:
  num_epochs: 50
  epoch_length: 128
  checkpoint_every: 500
  validate_every: 1
  resume_from: null
  optimizer:
    cls: torch.optim.SGD
    params:
      lr: 0.1
      momentum: 0.9
      weight_decay: 0.0001
      nesterov: false
  supervised_criterion:
    cls: torch.nn.CrossEntropyLoss
  lr_scheduler:
    cls: torch.optim.lr_scheduler.CosineAnnealingLR
    params:
      eta_min: 0.0
      T_max: null
  unsupervised_criterion:
    cls: torch.nn.CrossEntropyLoss
    params:
      reduction: none
ssl:
  num_train_samples_per_class: 25
  confidence_threshold: 0.7
  lambda_u: 1.0
  mu_ratio: 7
  cta_update_every: 15
name: fixmatch
seed: 543
debug: false
model: resnet18
num_classes: 10
ema_decay: 0.9
distributed:
  backend: xla-tpu
  nproc_per_node: 8
online_exp_tracking:
  wandb: 'true'



In [9]:
model, ema_model, optimizer, sup_criterion, lr_scheduler = utils.initialize(cfg)

In [10]:
lr_scheduler

{'cls': 'torch.optim.lr_scheduler.CosineAnnealingLR', 'params': {'eta_min': 0.0, 'T_max': None}, 'T_max': 6400}

In [11]:
optimizer

{'cls': 'torch.optim.SGD', 'params': {'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0001, 'nesterov': False}}

In [6]:
%cd /content

/content


### Let's train ResNet18 model in a faster mode

In [4]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=50 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1

/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'ssl/cta_pseudo': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'solver/default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'dataflow/cifar10': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
2021-06-17 04:31:00,037 ignite.distributed.launcher.Parallel INFO: I

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true